In [2]:
import  pandas as pd
import numpy as np
df = pd.read_csv('merged_df_abstract_preprocess_all.csv')
df = df.dropna(subset=['processed_abstract'])

C:\Users\cshch\AppData\Local\Temp\ipykernel_22496\2488393254.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_df_abstract_preprocess_all.csv')


In [4]:
print(df.shape)
print(df.columns)

(311557, 20)
Index(['Title', 'Abstract', 'Disciplines', 'Keywords', 'Source',
       'Disciplines_split', 'Astronomy', 'Biology', 'Chemistry',
       'Computer Science', 'Economics', 'Education & instruction',
       'Electrical Engineering', 'European & international law', 'History',
       'Mathematics', 'Physics', 'Social Sciences', 'Statistics',
       'processed_abstract'],
      dtype='object')


In [8]:
# !pip install --upgrade scikit-learn imbalanced-learn

  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.12.2
    Uninstalling imbalanced-learn-0.12.2:
      Successfully uninstalled imbalanced-learn-0.12.2


In [4]:
discipline_mapping = {
    "Astronomy": "Physics",
    "Astronomy/Physics": "Physics",
    "Biology": "Biology",
    "Chemistry/Physics": "Physics",
    "Computer Science": "Computer Science",
    "Computer Science/Electrical Engineering": "Computer Science",
    "Computer Science/Mathematics": "Mathematics",
    "Computer Science/Statistics": "Mathematics",
    "Economics": "Social Sciences",
    "Education & instruction": "Social Sciences",
    "Electrical Engineering": "Electrical Engineering",
    "European & international law": "Social Sciences",
    "History": "Social Sciences",
    "Mathematics": "Mathematics",
    "Mathematics/Physics": "Physics",
    "Mathematics/Statistics": "Mathematics",
    "Physics": "Physics",
    "Social Sciences": "Social Sciences",
    "Statistics": "Mathematics",
    "Computer Science/Electrical Engineering/Mathematics": "Mathematics"
}
# Apply the discipline mapping to the 'Disciplines' column in the DataFrame
df['Disciplines_merged'] = df['Disciplines'].map(discipline_mapping)

# Check the changes
print(df['Disciplines_merged'].value_counts())

Disciplines_merged
Computer Science          133914
Physics                    96677
Mathematics                68196
Social Sciences             5666
Electrical Engineering      5583
Biology                     1521
Name: count, dtype: int64


In [5]:
# Preprocessing
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Vectorize the Text Data using TF-IDF
vectorizer_Logistic_MultiDiscipline = TfidfVectorizer()
X = vectorizer_Logistic_MultiDiscipline.fit_transform(df['processed_abstract'])
y = df['Disciplines_merged']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Oversampling using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Before oversampling:")
print(y_train.value_counts())

print("After oversampling:")
print(pd.Series(y_train_resampled).value_counts())


Before oversampling:
Disciplines_merged
Computer Science          107030
Physics                    77448
Mathematics                54558
Electrical Engineering      4504
Social Sciences             4476
Biology                     1229
Name: count, dtype: int64
After oversampling:
Disciplines_merged
Computer Science          107030
Physics                   107030
Mathematics               107030
Social Sciences           107030
Biology                   107030
Electrical Engineering    107030
Name: count, dtype: int64


In [8]:
# 2 Logistic_MultiDisciplines
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

param_grid = {
    # 'C': [0.01, 0.1, 1, 10, 100, 1000],
    'C': [1000, 10000, 100000], # 正则化强度
    # 'penalty': ['l1', 'l2'],                      # 正则化类型
    # 'solver': ['liblinear', 'lbfgs', 'saga'],     # 求解器
    'max_iter': [1000]
}

model_Logistic_MultiDiscipline_oversampling = LogisticRegression()
grid_search = GridSearchCV(estimator=model_Logistic_MultiDiscipline_oversampling, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_resampled, y_train_resampled)

# 输出最佳参数和最佳得分
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# 使用最佳参数进行预测
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# 评估模型性能
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters found:  {'C': 1000, 'max_iter': 1000}
Best cross-validation score:  0.9668286150300538
Accuracy: 0.9061176017460522
Classification Report:
                         precision    recall  f1-score   support

               Biology       0.81      0.74      0.78       292
      Computer Science       0.94      0.91      0.93     26884
Electrical Engineering       0.34      0.56      0.42      1079
           Mathematics       0.86      0.87      0.86     13638
               Physics       0.96      0.95      0.95     19229
       Social Sciences       0.72      0.82      0.77      1190

              accuracy                           0.91     62312
             macro avg       0.77      0.81      0.79     62312
          weighted avg       0.91      0.91      0.91     62312



In [7]:
# Save model
import joblib
import os
save_dir = 'model'
os.makedirs(save_dir, exist_ok=True)

model_file_path = os.path.join(save_dir, 'best_Model_Logistic_MultiDiscipline_oversampling.joblib')
joblib.dump(best_model, model_file_path)

loaded_model = joblib.load(model_file_path)
print("Model successfully loaded:", loaded_model)

In [15]:
import pandas as pd

n_top_keywords = 100
feature_names = vectorizer_Logistic_MultiDiscipline.get_feature_names_out()

keywords_dict = {}

for i, label in enumerate(model_Logistic_MultiDiscipline_oversampling.classes_):
    coef = model_Logistic_MultiDiscipline_oversampling.coef_[i]
    top_keywords_idx = coef.argsort()[-n_top_keywords:][::-1]

    top_keywords = [(feature_names[idx], coef[idx]) for idx in top_keywords_idx]
    keywords_dict[label] = top_keywords

keywords_df = pd.DataFrame.from_dict(keywords_dict, orient='index')
keywords_df.to_csv("keywords_from_model/2_keywords_Logistic_oversampling.csv")
